In [2]:
from tensorflow import keras
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from scipy.optimize import fmin_l_bfgs_b
import matplotlib.pyplot as plt
# from scipy.misc import imsave
from tensorflow.keras.preprocessing import image_dataset_from_directory
import time
# import cv2 as cv
from PIL import Image

from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [3]:
#load trained model
FCN_original = keras.models.load_model('C:/Users/MrLin/Documents/Experiments/Fine_Scale_Autoencoder_Lichen/FCN512_savemodel.h5')

def process_image(imgtensor):
    imgtensor *= (1/255)
    return imgtensor

def deprocess_image(imgtensor):
    imgtensor *= 255
    imgtensor = np.clip(imgtensor, 0, 255)
    imgtensor = np.uint8(imgtensor)
    return imgtensor

FCN_original.summary()

Model: "FCN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
FCN_input (InputLayer)       [(None, 512, 512, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 512, 512, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 512, 512, 32)      9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 512, 512, 32)      128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 512, 512, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 512, 512, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 512, 512, 64)      256     

In [7]:
# plot reconstructions

datadir = 'C:/Users/MrLin/Documents/Experiments/Fine_Scale_Autoencoder_Lichen/Results/test_images/input' #your images must be inside a subfolder in the last folder of datadir. if using multiple classes, put each class in one subfolder of the train_data folder. also put validation and test sets each in their own subfolder at the same level as the training folder
#create images dataset
img_dataset_validation = image_dataset_from_directory(
    datadir, shuffle = False, image_size=(512, 512), batch_size=1, color_mode='rgb', validation_split=None, label_mode=None)

#rescale pixel values
img_dataset_validation = img_dataset_validation.map(process_image)
print(img_dataset_validation.cardinality()) #set this value as the number of validation steps in .fit

#PREDICT images 
xhat = FCN_original.predict(img_dataset_validation, steps=18)

xhat = deprocess_image(xhat)
# xhat = xhat.astype(np.uint8)
xhat.shape
for j in range(18):
    pil_im = Image.fromarray(xhat[j,:,:,:],mode='RGB') # convert each slice of predicted batch to a PIL image object  
    pil_im.save('C:/Users/MrLin/Documents/Experiments/Fine_Scale_Autoencoder_Lichen/Results/test_images/reconstructions/New folder/pred_' + f'{j+0:02}' + '.png')


Found 18 files belonging to 1 classes.
tf.Tensor(18, shape=(), dtype=int64)


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node FCN/conv2d/Relu (defined at <ipython-input-7-2c01ae5a9457>:13) ]] [Op:__inference_predict_function_1390]

Function call stack:
predict_function


In [9]:
# creates a dict of models/layers that can be easily accessed to get intermediate outputs
outputs_dict = dict([(layer.name, layer.output) for layer in FCN_original.layers[1:]]) #we wont need the output of input layer
outputs_dict

{'conv2d': <tf.Tensor 'conv2d/Relu:0' shape=(None, 512, 512, 32) dtype=float32>,
 'conv2d_1': <tf.Tensor 'conv2d_1/Relu:0' shape=(None, 512, 512, 32) dtype=float32>,
 'batch_normalization': <tf.Tensor 'batch_normalization/cond/Identity:0' shape=(None, 512, 512, 32) dtype=float32>,
 'conv2d_2': <tf.Tensor 'conv2d_2/Relu:0' shape=(None, 512, 512, 64) dtype=float32>,
 'conv2d_3': <tf.Tensor 'conv2d_3/Relu:0' shape=(None, 512, 512, 64) dtype=float32>,
 'batch_normalization_1': <tf.Tensor 'batch_normalization_1/cond/Identity:0' shape=(None, 512, 512, 64) dtype=float32>,
 'conv2d_4': <tf.Tensor 'conv2d_4/Relu:0' shape=(None, 512, 512, 64) dtype=float32>,
 'conv2d_5': <tf.Tensor 'conv2d_5/Relu:0' shape=(None, 512, 512, 64) dtype=float32>,
 'batch_normalization_2': <tf.Tensor 'batch_normalization_2/cond/Identity:0' shape=(None, 512, 512, 64) dtype=float32>,
 'conv2d_6': <tf.Tensor 'conv2d_6/Relu:0' shape=(None, 512, 512, 64) dtype=float32>,
 'conv2d_7': <tf.Tensor 'conv2d_7/Relu:0' shape=(None

In [41]:
# EXTRACT GLOBAL AVG POOLING BASED FEATURE VECTORS FOR EACH IMAGE AND STORE IN AN ARRAY
# datadir = 'D:/Datasets/Lichen_partitioned' #your images must be inside a subfolder in the last folder of datadir. if using multiple classes, put each class in one subfolder of the train_data folder. also put validation and test sets each in their own subfolder at the same level as the training folder
datadir = 'D:/Datasets/Lichen_partitioned_manualclusters'
#create images dataset
img_dataset = image_dataset_from_directory(
    datadir, shuffle = False, image_size=(512, 512), batch_size=1, color_mode='rgb', validation_split=None, label_mode=None)

def get_GAP(layer_output):
    return K.mean(K.batch_flatten(K.permute_dimensions(layer_output,(2,0,1))), axis=1)
    
GAP_vector = []
# cycle through all non-batch norm layers and concat the GAP into one long vector with length equal to the number of channels in the entire network
for name in outputs_dict:
    if name.find('batch_normalization') == -1: # skip batch norm layers
        layer_output = outputs_dict[name]
        GAP_vector.append(get_GAP(layer_output[0]))
    
GAP_vector = K.concatenate(GAP_vector, axis=0)
    
#create K.function mapping the input img to GAP feature vector
inp = FCN_original.input
image2GAP_vector = K.function([inp], GAP_vector) #

Nimgs = 203 #4322
img_dataset_sample = img_dataset.take(Nimgs)
dat = np.zeros(shape=(Nimgs,451))
j=0
for img in img_dataset_sample:
    print(j)
    GAP_j = image2GAP_vector([img]) # the first moment a concrete array enters 
    dat[j,:]=GAP_j
    j+=1
# dat = np.array(dat)
# dat
dat.shape

Found 203 files belonging to 1 classes.
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202


(203, 451)

In [50]:
# embed GAP vectors in 2D
from sklearn.manifold import TSNE
p_data_emb = TSNE(n_components=2, 
                  perplexity=5.0, 
                  early_exaggeration=24.0, 
                  learning_rate=200.0, 
                  n_iter=1000, 
                  n_iter_without_progress = 51,
                  metric='euclidean', 
                  init='pca', 
                  verbose=1).fit(dat)
p_data_emb.embedding_.shape
emb = p_data_emb.embedding_ #if using TSNE

[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 203 samples in 0.000s...
[t-SNE] Computed neighbors for 203 samples in 0.017s...
[t-SNE] Computed conditional probabilities for sample 203 / 203
[t-SNE] Mean sigma: 331.952780
[t-SNE] KL divergence after 250 iterations with early exaggeration: 229.871994
[t-SNE] KL divergence after 1000 iterations: 1.302042


In [ ]:
# plot the embedding
import plotly.graph_objects as go
from PIL import Image
from os import listdir
images_dir = 'D:/Datasets/Lichen_partitioned_manualclusters/New folder'
file_list = listdir(images_dir)


fig2 = go.Figure(data=[go.Scatter(
    name='training images',
    x=emb[:,0],
    y=emb[:,1],
    mode='markers',
    marker=dict(
        size=4, color='black',symbol='circle')
)])

fig2.update_layout(
    width=2000,
    height=1800,
                
    scene_xaxis = dict(tickfont=dict(size=12),
                title_font_size=20),
    scene_yaxis = dict(tickfont=dict(size=12),
                title_font_size=20),
)

mini_image_size=700
for j, filename in enumerate(file_list):
    lichen = Image.open('D:/Datasets/Lichen_partitioned_manualclusters/New folder/' + filename)
    fig2.add_layout_image(
            dict(
                source=lichen,
                xref="x",
                yref="y",
                x=emb[j,0],
                y=emb[j,1],
                sizex=mini_image_size,
                sizey=mini_image_size,
                sizing="contain",
                opacity=1,
                layer="above")
    )

fig2.show()

In [40]:
# test code. (not needed for algorithm)
from os import listdir
images_dir = 'D:/Datasets/Lichen_partitioned_manualclusters/New folder'
file_list = listdir(images_dir)

for filename in file_list:
    print('D:/Datasets/Lichen_partitioned_manualclusters/New folder/' + filename)

D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_1.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_10.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_11.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_12.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_13.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_14.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_15.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_16.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_17.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_18.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_19.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_2.png
D:/Datasets/Lichen_partitioned_manualclusters/New folder/pat_I145_20.png
D:/Datasets/Lichen_partitioned_manualclusters/New fol